# MLC v2

In [11]:
import numpy as np

# The Equation for Backpropagation
$$\begin{array}{cll}
\delta^L & = & \nabla_a C \odot \sigma'(z^L) \\ 
\delta^l & = & ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l) \\
\frac{\partial C}{\partial b^l_j} & = & \delta^l_j \\
\frac{\partial C}{\partial w^l_{jk}} & = & a_k^{l-1} \delta^l_j
\end{array}$$

# Layer Class

In [58]:
class Layer:
    def __init__(self, input_size, output_size, activation):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)
        self.activate, self.d_activate = activation
    def forward(self, x):
        self.inputs = x
        self.outputs = self.activate(x @ self.weights + self.bias)
        return self.outputs
    # Returns the delta for this layer
    def err(self, delta_p, w_p):
        #print(f"delta: {delta_p}")
        delta = (delta_p @ w_p.T) * self.d_activate(self.outputs)
        #delta = (w_p @ delta_p.T) * self.d_activate(self.outputs)
        return delta
    # Perform backpropagation, returning the weights and bias gradients for this layer
    def back(self, delta):
        #print(f"self.inputs: {self.inputs.T}")
        #print(f"delta input: {delta}")
        #print(f"delta: {delta}")
        #print(f"self.inputs.T: {self.inputs.T}")
        
        grad_b = delta
        grad_w =  self.inputs.T @ delta 

        return (grad_w, grad_b)

# Network 

In [86]:
class Network:
    def __init__(self, loss, d_loss):
        self.layers = []
        self.loss = loss
        self.d_loss = d_loss
    def add(self, layer):
        self.layers.append(layer)
    def fit(self, inputs, labels, learn_rate):
        # TODO: rename all mentions of epoch here to batch
        epoch_loss = 0 
        grad_w = []
        grad_b = []
        for x in enumerate(inputs):
            output = x[1]
            # forward propagate
            for layer in self.layers:
                output = layer.forward(output)

            epoch_loss += self.loss(output, labels[x[0]])
            
            lg_w = [] # gradients are stored in reverse of layers
            lg_b = []
            # backpropagate
            #delta = self.d_loss(output, labels[x[0]]) *  self.layers[-1].d_activate(output)
            delta = self.d_loss(output, labels[x[0]]) * self.layers[-1].d_activate(output) 
            #print(f"initial delta: {delta}")
            #print(f"# of layers: {len(self.layers)}")
            for i in range(len(self.layers)-1, -1, -1):
                #print(f"======At layer {i}")
                gw, gb = self.layers[i].back(delta)
                lg_w.append(gw)
                lg_b.append(gb)
                if i-1 >= 0:
                    delta = self.layers[i-1].err(delta, layer.weights)
            grad_w.append(lg_w)
            grad_b.append(lg_b)

        # Sum and average
        #grad_w = np.add.reduce(grad_w) / len(inputs)
        #grad_b = np.add.reduce(grad_b) / len(inputs)
        #print(f"grad_b: {grad_b}")
        grad_b_avg = []
        for i in range(len(grad_b[0])):
            b0 = grad_b[0][i]
            for j in range(1, len(grad_b)):
                #print(f"+=+= {grad_b[j][i]}")
                b0 += grad_b[j][i]
            grad_b_avg.append(b0 / len(grad_b))

        #print(f"grad_w: {grad_w}")
        grad_w_avg = []
        for i in range(len(grad_w[0])):
            w0 = grad_w[0][i]
            for j in range(1, len(grad_w)):
                #print(f"+=+= {grad_w[j][i]}")
                w0 += grad_w[j][i]
            #print(f"number of grad_ws: {len(grad_w)}")
            grad_w_avg.append(w0 / len(grad_w))
        #print(f"grad_w_avg: {grad_w_avg}")
            
        
        #print(f"grad_w: {grad_w}")
        #print(f"grad_b: {grad_b}")
        for i in range(len(self.layers)-1, -1, -1):
            layer = self.layers[i]
            #print(f"layer weights: {layer.weights}")
            #print(f"grad_w_avg: {grad_w_avg}")
            layer.weights -= learn_rate * grad_w_avg[-i+1]
            layer.bias -= learn_rate * grad_b_avg[-i+1]
        
        # print(f"batch loss: {epoch_loss / len(inputs)}")
        return epoch_loss / len(inputs)
            

# Activator functions

In [87]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))
    
def d_sigmoid(x: np.ndarray) -> np.ndarray:
    return np.exp(-x) / ((1 + np.exp(-x))**2)

# Loss functions

In [88]:
def mse(dy, y):
    return np.mean(np.power(dy-y, 2));

def mse_prime(dy, y):
    return 2*(dy-y)/dy.size;

def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

# Batchtize

In [89]:
from random import randint

def batchtize(inputs, labels, batch_size):
    if len(inputs) != len(labels):
        raise
    if len(inputs) % batch_size != 0:
        raise
        
    c = 0
    chosen_indexes = []
    total_x = []
    total_y = []
    batch_x = []
    batch_y = []
    while c < len(inputs):
        i = randint(0, len(inputs)-1)
        while i in chosen_indexes:
            i = randint(0, len(inputs)-1)
        chosen_indexes += [i]
        if c != 0 and c % batch_size == 0:
            total_x += batch_x
            total_y += batch_y
            batch_x = []
            batch_y = []
        #print(f"inserting input: {inputs[i]}")
        batch_x.append(inputs[i])
        batch_y.append(labels[i])
        c+= 1
        #print("ran!")

    total_x += batch_x
    total_y += batch_y
    
    return (total_x, total_y)
    

# Test Batchtize

In [90]:
x = [[1,0], [0,1], [1,1], [0,0]]
y = [[1], [1], [0], [0]]

print(batchtize(x, y, 1))

([[0, 1], [0, 0], [1, 1], [1, 0]], [[1], [0], [0], [1]])


# Example

In [92]:
net = Network(mse, mse_prime)
activator = (tanh, tanh_prime)

net.add(Layer(2, 3, activator))
net.add(Layer(3, 1, activator))

#x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
#y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
#net.fit(x_train, y_train, epochs=1000, learn_rate=0.1)

# training data
x_train = [np.array([[0,0]]), np.array([[0,1]]), np.array([[1,0]]), np.array([[1,1]])]
y_train = [np.array([[0]]), np.array([[1]]), np.array([[1]]), np.array([[0]])]
#net.fit(x_train, y_train, epochs=1000, learn_rate=0.1)

epochs = 1000
batch_size = 1
for i in range(epochs):
    print(f"+ Epoch: {i}")
    x_b, y_b = batchtize(x_train, y_train, batch_size)
    sum_batch_loss = 0
    for i in range(len(x_b)):
        #print(f"x_b[i]: {x_b}")
        sum_batch_loss += net.fit([x_train[i]], [y_train[i]], learn_rate=0.1)
    print(f"    loss: {sum_batch_loss / batch_size}")


+ Epoch: 0
    loss: 1.78735883134589
+ Epoch: 1
    loss: 1.6062214782858293
+ Epoch: 2
    loss: 1.4157822742254607
+ Epoch: 3
    loss: 1.3152196871047184
+ Epoch: 4
    loss: 1.2983525322490705
+ Epoch: 5
    loss: 1.2994825176058038
+ Epoch: 6
    loss: 1.2973195257176175
+ Epoch: 7
    loss: 1.2927437056565432
+ Epoch: 8
    loss: 1.2876655306375806
+ Epoch: 9
    loss: 1.282794548036746
+ Epoch: 10
    loss: 1.278294228683814
+ Epoch: 11
    loss: 1.274157842199099
+ Epoch: 12
    loss: 1.2703380515402116
+ Epoch: 13
    loss: 1.2667850961427534
+ Epoch: 14
    loss: 1.2634561502370563
+ Epoch: 15
    loss: 1.260316334029151
+ Epoch: 16
    loss: 1.2573376414240551
+ Epoch: 17
    loss: 1.2544975926840716
+ Epoch: 18
    loss: 1.251778069971272
+ Epoch: 19
    loss: 1.249164403012274
+ Epoch: 20
    loss: 1.2466446719738782
+ Epoch: 21
    loss: 1.2442091788062428
+ Epoch: 22
    loss: 1.2418500442498872
+ Epoch: 23
    loss: 1.2395608972171812
+ Epoch: 24
    loss: 1.2373366316